### [PyMongo Tutorial](http://api.mongodb.org/python/current/tutorial.html)

In [3]:
from pymongo import MongoClient

# connect on the default host and post
client = MongoClient('localhost', 27017)
# or 
# client = MongoClinet('mongodb://localhost:27017/')

# Get a database
db = client.test_database
# or
# db = client['test-databse'] if the name is 'test-database'

# Get a collection (a group of documents)
# roughly as the equivalent of a table in relational database
collection = db.test_collection
# or collection = db['test-collection']
# Note that, collections and databases in MongoDB is created lazily.
# Noe of the above commands have actually performed any operations on
# the MongoDb server.

Data in MongoDB is represent using JSON-stye documents.

In [4]:
import datetime
post = {"author": "Mike",
       "text": "My first blog post!",
       "tags": ["mongodb", "python", "pymongo"],
       "date": datetime.datetime.utcnow()}

# Insert a document into a collection
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('56f3397725bb2a09d5db9cea')

When a document is inserted, a special key `_id` is automatically added if the document doesn't contain an `_id` key. `_id` is unique across the collection. 

In [5]:
db.collection_names(include_system_collections=False)

[u'posts']

In [6]:
# get certain document
posts.find_one()

{u'_id': ObjectId('56f3397725bb2a09d5db9cea'),
 u'author': u'Mike',
 u'date': datetime.datetime(2016, 3, 24, 0, 48, 55, 882000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}

In [7]:
# query specific elements
posts.find_one({"author": "Mike"})

{u'_id': ObjectId('56f3397725bb2a09d5db9cea'),
 u'author': u'Mike',
 u'date': datetime.datetime(2016, 3, 24, 0, 48, 55, 882000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}

In [8]:
# Query by ObjectID
post_id

ObjectId('56f3397725bb2a09d5db9cea')

In [9]:
posts.find_one({"_id": post_id})

{u'_id': ObjectId('56f3397725bb2a09d5db9cea'),
 u'author': u'Mike',
 u'date': datetime.datetime(2016, 3, 24, 0, 48, 55, 882000),
 u'tags': [u'mongodb', u'python', u'pymongo'],
 u'text': u'My first blog post!'}

In [10]:
# Note taht an ObjectID is not the same as its string representation
post_id_as_str = str(post_id)
posts.find_one({"_id": post_id_as_str}) # no results

A common task in web applications is to get an ObjectId from the request URL and find the matching document. It’s necessary in this case to convert the ObjectId from a string before passing it to find_one:

In [11]:
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

In [12]:
# Bulk insert
new_posts = [{"author": "Mike",
             "text": "Another post!",
             "tags": ["bulk", "insert"],
             "date": datetime.datetime(2009, 11, 12, 11, 14)},
            {"author": "Eliot",
            "title": "MongoDB is fun",
            "text": "and pretty easy too!",
            "date": datetime.datetime(2009, 11, 10, 10, 45)}]
result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('56f33c9925bb2a09d5db9ceb'), ObjectId('56f33c9925bb2a09d5db9cec')]

In [14]:
# Query more than one document
for post in posts.find():
    print post

{u'date': datetime.datetime(2016, 3, 24, 0, 48, 55, 882000), u'text': u'My first blog post!', u'_id': ObjectId('56f3397725bb2a09d5db9cea'), u'author': u'Mike', u'tags': [u'mongodb', u'python', u'pymongo']}
{u'date': datetime.datetime(2009, 11, 12, 11, 14), u'text': u'Another post!', u'_id': ObjectId('56f33c9925bb2a09d5db9ceb'), u'author': u'Mike', u'tags': [u'bulk', u'insert']}
{u'date': datetime.datetime(2009, 11, 10, 10, 45), u'text': u'and pretty easy too!', u'_id': ObjectId('56f33c9925bb2a09d5db9cec'), u'author': u'Eliot', u'title': u'MongoDB is fun'}


In [15]:
for post in posts.find({"author": "Mike"}):
    print post

{u'date': datetime.datetime(2016, 3, 24, 0, 48, 55, 882000), u'text': u'My first blog post!', u'_id': ObjectId('56f3397725bb2a09d5db9cea'), u'author': u'Mike', u'tags': [u'mongodb', u'python', u'pymongo']}
{u'date': datetime.datetime(2009, 11, 12, 11, 14), u'text': u'Another post!', u'_id': ObjectId('56f33c9925bb2a09d5db9ceb'), u'author': u'Mike', u'tags': [u'bulk', u'insert']}


In [16]:
posts.count()

3

In [17]:
posts.find({"author": "Mike"}).count()

2

In [18]:
# Range Queries
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    print post

{u'date': datetime.datetime(2009, 11, 10, 10, 45), u'text': u'and pretty easy too!', u'_id': ObjectId('56f33c9925bb2a09d5db9cec'), u'author': u'Eliot', u'title': u'MongoDB is fun'}
{u'date': datetime.datetime(2009, 11, 12, 11, 14), u'text': u'Another post!', u'_id': ObjectId('56f33c9925bb2a09d5db9ceb'), u'author': u'Mike', u'tags': [u'bulk', u'insert']}


In [20]:
# Indexing
import pymongo
result = db.profiles.create_index([("user_id", pymongo.ASCENDING)],
                                 unique=True)
list(db.profiles.index_information())

[u'user_id_1', u'_id_']

Notice that we have two indexes now: one is the index on `_id` that MongoDB creates automatically, and the other is the index on `user_id` we just created.

In [21]:
# set up some user profiles
user_profiles = [
    {'user_id': 211, 'name': 'Luke'},
    {'user_id': 212, 'name': 'Ziltoid'}]
result = db.profiles.insert_many(user_profiles)